# Lesson 3: Implementing Error Handling and Retries with Python Decorators

# Implementing Error Handling and Retries

Hello, and welcome back! Last time, we successfully made our first Whisper API request to transcribe audio using OpenAI's service. Armed with that knowledge, we'll now build resilience into your transcription system by implementing error handling and adding retries. This lesson expands on your current skills to ensure that even when errors occur, your application remains robust and continues running smoothly.

In this lesson, you'll learn how to use Python decorators to wrap function calls for error handling, implement retries for more reliable API requests, and deepen your understanding of adding functionality to functions. These concepts are critical when interacting with APIs since network issues or server timeouts shouldn't derail your entire application.

## Understanding Implementing Error Handling and Retries

In real-world applications, errors can arise due to various reasons, such as network interruptions, server downtimes, or temporary glitches. Instead of terminating the process, implementing retries allows the system to recover gracefully.

Python decorators play a vital role here. A decorator is a design pattern in Python that allows you to add new functionality to an existing object — in this case, a function — without modifying its structure. It’s like putting a flexible wrapper around a function that you can use to introduce additional behavior like logging, restricting access, or retry mechanisms.

## Implementing Error Handling in Transcription

Let's break down the given example where we utilize a decorator for error handling:

```python
import time
from functools import wraps
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

def retry_on_error(max_retries=3, delay=5):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    retries += 1
                    if retries == max_retries:
                        raise
                    print(f"Error: {e}. Retrying in {delay} seconds...")
                    time.sleep(delay)
            return None
        return wrapper
    return decorator
```

### Here’s how it works step-by-step:

1. **Decorator Definition**: `retry_on_error` is defined, taking `max_retries` and `delay` as arguments. These control how many attempts occur and the wait time between them.
2. **Inner Function Decorator**: Within `retry_on_error`, another function decorator is declared, which will wrap the function you plan to retry on error. The `wraps` decorator from `functools` maintains the original function's metadata.
3. **Error Handling**: The wrapper nested within counts retries. It calls the original function (`func`) inside a try-except block. If an exception arises, it waits for `delay` seconds before retrying.
4. **Final Attempt and Failure**: If the maximum retries are exceeded and failure persists, the exception is re-raised to signal a lasting issue.

## Applying the Decorator to the Transcription Function

The `retry_on_error` decorator is then applied to the `transcribe_audio` function:

```python
@retry_on_error(max_retries=3, delay=5)
def transcribe(audio_file):
    """Transcribe a small audio file using OpenAI Whisper API"""
    try:
        with open(audio_file, "rb") as audio:
            response = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio,
                timeout=60
            )
            return response.text
    except Exception as e:
        print(f"Error during transcription: {e}")
        return None

if __name__ == "__main__":
    result = transcribe("resources/sample_audio.mp3")
    print("Transcription:", result)
```

This function attempts to read and transcribe audio using the Whisper API. In case of an exception (like a network issue), the retry logic kicks in, reattempting the transcription up to three times with five seconds between each try. If transcription fails even after retries, a message is displayed, and `None` is returned.

## Lesson Summary

In this lesson, we've enhanced your transcription system by implementing error handling and retries using Python decorators. We've learned that errors in real-world applications can arise due to various factors like network interruptions or server downtimes. Instead of terminating processes, retries enable the system to recover gracefully. Python decorators allow us to wrap functions to add new functionality, such as retry mechanisms, without altering their structure.

We demonstrated the use of a `retry_on_error` decorator, which can handle retries and delays between attempts. The decorator was applied to a transcription function that utilizes the Whisper API. This function attempts to transcribe audio, retrying up to three times with five seconds between attempts in case of errors, ensuring reliability and robustness.

Incorporating error handling with retries is crucial in software development, particularly when interacting with APIs. This robust pattern enhances user experience and system stability by addressing transient failures, making your application resilient and reliable in production environments.


## Enhance Function with Retry Mechanism Using Decorators

Hey, Space Explorer!

Let's see how we can enhance your Python function with retries and failure handling using Python decorators. Watch as our function fails on the first two attempts but succeeds on the third try thanks to our retry mechanism!

Run the code to see how it works!

```python
from functools import wraps
import time


def retry_on_exception(max_attempts=3, wait_time=2):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    return func(*args, **kwargs)
                except Exception as error:
                    attempts += 1
                    print(f"Attempt {attempts} failed: {error}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
            return None
        return wrapper
    return decorator


# Counter to track number of function calls
attempt_counter = 0


# Example function using the decorator for retry logic
@retry_on_exception(max_attempts=3, wait_time=2)
def fetch_data():
    global attempt_counter
    attempt_counter += 1
    if attempt_counter < 3:
        raise ValueError(f"Request failed on attempt {attempt_counter}!")
    return "Data fetched successfully"


# Execute the function once, it will retry automatically
response = fetch_data()
print(response)

```

## Add Error Handling and Retry Mechanism to Transcription Function

Hey, Space Explorer!

In this task, we're going to enhance a Python function that checks whether a random number is greater than 8 with retries and failure handling using Python decorators. Your task is to complete the decorator and see how the function retries up to ten times before succeeding or stopping!

Implement the missing parts of the decorator in the starter code and run it to observe the retry mechanism in action

```python
from functools import wraps
import time
import random


def retry_on_exception(max_attempts, wait_time):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
             # TODO: implement the retry logic
        return wrapper
    return decorator


# Example function using the decorator for retry logic
@retry_on_exception(max_attempts=10, wait_time=1)
def check_random_number():
    num = random.randint(1, 10)
    if num <= 8:
        raise ValueError(f"Number {num} is not greater than 8.")
    return f"Number {num} is greater than 8."


# Execute the function once, it will retry automatically
response = check_random_number()
print(response)


```

Here’s the complete fixed code with the retry mechanism implemented:

```python
from functools import wraps
import time
import random


def retry_on_exception(max_attempts, wait_time):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    # Attempt to run the function
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    print(f"Attempt {attempts} failed with error: {e}")
                    if attempts >= max_attempts:
                        print(f"Maximum attempts reached. Failing.")
                        raise e
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
        return wrapper
    return decorator


# Example function using the decorator for retry logic
@retry_on_exception(max_attempts=10, wait_time=1)
def check_random_number():
    num = random.randint(1, 10)
    if num <= 8:
        raise ValueError(f"Number {num} is not greater than 8.")
    return f"Number {num} is greater than 8."


# Execute the function once, it will retry automatically
response = check_random_number()
print(response)
```

This will retry up to 10 times if the generated random number is not greater than 8, with a 1-second wait between retries. The output will print the attempt number and any error that occurs during the retries. Once a number greater than 8 is generated, the function will return success.

You're tasked with building a robust transcription function using Python decorators for error handling and retries. Implement the function according to the plan using the Whisper API, with retry logic. Use random failures to simulate real-life transcription challenges, and ensure your application remains stable by retrying operations. Good luck, Space Explorer!


```python
from functools import wraps
import time
import random
from openai import OpenAI

client = OpenAI()


# TODO: Implement error handling and retrying decorator

# TODO: Apply decorator for the `transcribe` method
# to retry up to 3 times, with a delay of 1 second between retries


def transcribe(file_path):
    """
    Transcribe an audio file using OpenAI's API.
    """
    if random.randint(1, 2) == 1:
        raise ValueError("Unable to process transcription at that point")
    try:
        with open(file_path, 'rb') as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
            return transcript.text
    except Exception as e:
        raise Exception(f"Transcription failed: {str(e)}")


```

To implement error handling and retry logic in your transcription function using decorators, we’ll add a `retry_on_exception` decorator to handle retries on failure. The decorator will attempt the `transcribe` function up to 3 times with a 1-second delay between retries. 

Here’s how you can implement the retry logic and apply the decorator to the `transcribe` function:

### Full Implementation:

```python
from functools import wraps
import time
import random
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI()

# Retry decorator
def retry_on_exception(max_attempts, wait_time):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    # Attempt to run the function
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    print(f"Attempt {attempts} failed with error: {e}")
                    if attempts >= max_attempts:
                        print(f"Maximum attempts reached. Failing.")
                        raise e
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
        return wrapper
    return decorator

# Apply decorator for retry logic to the transcribe function
@retry_on_exception(max_attempts=3, wait_time=1)
def transcribe(file_path):
    """
    Transcribe an audio file using OpenAI's Whisper API.
    """
    if random.randint(1, 2) == 1:  # Simulating a random failure
        raise ValueError("Unable to process transcription at that point")
    
    try:
        with open(file_path, 'rb') as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
            return transcript.text
    except Exception as e:
        raise Exception(f"Transcription failed: {str(e)}")

# Example of calling the transcribe function
file_path = "path_to_audio_file.mp3"  # Update with an actual file path
try:
    result = transcribe(file_path)
    print(result)
except Exception as e:
    print(f"Final failure: {e}")
```

### Explanation:

1. **`retry_on_exception` Decorator**: This decorator wraps the `transcribe` function and retries the operation if an exception occurs. It will retry up to `max_attempts` (3 times in this case), with a delay of `wait_time` (1 second) between retries.

2. **Transcription Function**: The `transcribe` function simulates a random failure (via `random.randint(1, 2) == 1`) to mimic real-life challenges in transcription, such as network issues. If no error occurs, the transcription request is sent to OpenAI's Whisper API to process the audio file.

3. **File Handling**: The `transcribe` function opens an audio file (the file path must be valid for your setup) and uses OpenAI's Whisper API to transcribe it.

4. **Error Handling and Retry**: If the `transcribe` function fails (either due to the random failure or an exception from the Whisper API), the decorator retries the operation up to 3 times. After reaching the maximum retry attempts, it raises the final exception.

This implementation ensures that your application can handle temporary failures and retry operations, providing a more robust transcription service.

Let me know if you need further adjustments!